# Other get options

#### Sections
* [Post-processing options](#Post-processing-options)
* [`tzout`](#tzout)
* [`fill`](#fill)
* [`include` and `exclude`](#include-and-exclude)
* [`side`](#side)
* [`close_only`](#close_only)
* [`lose_single_symbol`](#lose_single_symbol)

#### Note

The cell **outputs** shown in this tutorial are based on executing the cells at the time shown in the output of the following cell. Simply rerun the cells to bring any dynamic output up to date.

In [2]:
import pandas as pd
now = pd.Timestamp.now(tz="UTC").floor("T")
print(f"{now!r}")
print(f"{now.astimezone('America/New_York')!r}")

Timestamp('2022-05-13 12:24:00+0000', tz='UTC')
Timestamp('2022-05-13 08:24:00-0400', tz='America/New_York')


## Setup

Run the following cell to import tutorial dependencies.

In [3]:
from market_prices import PricesYahoo
from market_prices.support import tutorial_helpers as th
from market_prices import helpers

Run the following cell to instantiate prices objects and define values used in this tutorial.

In [4]:
_prices = PricesYahoo("MSFT, 9988.HK, AZN.L", lead_symbol="MSFT")
xnys = _prices.calendar_default
xhkg = _prices.calendars["9988.HK"]
_sessions_range = th.get_sessions_range_for_bi(_prices, _prices.bis.T1)
length = pd.Timedelta(hours=18, minutes=30)
session = th.get_conforming_cc_sessions(_prices.cc, length, *_sessions_range, 1)[0]
start_H1 = th.get_sessions_range_for_bi(_prices, _prices.bis.H1)[0]
start_H1_oob = helpers.to_tz_naive(xnys.session_offset(start_H1, -2))

## Post-processing options

This tutorial covers `get` options available to **format and tidy** the price table.

The `get` method implements these options by calling the price table's `.pt.operate` method (see the [Tidying data](./pt_accessor.ipynb#Tidying-data) section of the [pt accessor](./pt_accessor.ipynb) tutorial). Rather than passing these options to `get`, the same effect could be achieved by subsequently passing the options to the `.pt.operate` method. The options are included to `get` simply as a convenience.

Be clear, these options are implemented as **post-processing operations** undertaken to the price table. They do not have any effect on the table interval, the period represented by each indice or the overall period covered by the price table.

The following tables include none of these options and will be used for the purpose of comparison.

In [5]:
prices = PricesYahoo("MSFT, 9988.HK, AZN.L", lead_symbol="MSFT")
session # for reference

Timestamp('2022-04-19 00:00:00')

In [6]:
kwargs_daily = dict(interval="1D", end=session, days=10)
df_daily = prices.get(**kwargs_daily)
df_daily

symbol            MSFT                                                \
                  open        high         low       close    volume   
2022-04-05  313.269989  314.869995  309.869995  310.880005  23156700   
2022-04-06  305.190002  307.000000  296.709991  299.500000  40110400   
2022-04-07  296.660004  303.649994  296.350006  301.369995  31411200   
2022-04-08  300.440002  301.119995  296.279999  296.970001  24361900   
2022-04-11  291.790009  292.609985  285.000000  285.260010  34569300   
2022-04-12  289.239990  290.739990  280.489990  282.059998  30966700   
2022-04-13  282.730011  288.579987  281.299988  287.619995  21907200   
2022-04-14  288.089996  288.309998  279.320007  279.829987  28221600   
2022-04-18  278.910004  282.459991  278.339996  280.519989  20778000   
2022-04-19  279.380005  286.170013  278.410004  285.299988  22297700   

symbol         9988.HK                                                  \
                  open        high         low       close      volume   
2022-04-05         NaN         NaN         NaN         NaN         NaN   
2022-04-06  109.099998  109.800003  107.099998  107.599998  46638063.0   
2022-04-07  106.400002  108.900002  105.199997  105.199997  35575706.0   
2022-04-08  102.800003  103.800003  101.500000  103.800003  38153586.0   
2022-04-11  102.800003  103.000000   97.849998   98.500000  40131164.0   
2022-04-12   98.500000  100.599998   96.349998   99.000000  39683531.0   
2022-04-13   98.400002   99.500000   97.000000   98.500000  24650087.0   
2022-04-14   97.650002   98.000000   93.849998   95.500000  52348941.0   
2022-04-18         NaN         NaN         NaN         NaN         NaN   
2022-04-19   92.550003   93.449997   90.900002   91.500000  34060706.0   

symbol        AZN.L                                                  
               open          high           low    close     volume  
2022-04-05  10112.0  10350.000000  10078.323242  10334.0  2529617.0  
2022-04-06  10242.0  10481.076172  10227.108398  10462.0  2251895.0  
2022-04-07  10554.0  10680.000000  10523.267578  10668.0  3462762.0  
2022-04-08  10734.0  11000.000000  10713.238281  10930.0  2542196.0  
2022-04-11  10888.0  10950.000000  10746.000000  10858.0  2561255.0  
2022-04-12  10720.0  10754.000000  10398.000000  10504.0  2791098.0  
2022-04-13  10608.0  10640.419922  10464.099609  10510.0  2102995.0  
2022-04-14  10432.0  10574.000000  10392.000000  10536.0  1980089.0  
2022-04-18      NaN           NaN           NaN      NaN        NaN  
2022-04-19  10478.0  10582.000000  10428.000000  10500.0  3260649.0

In [7]:
kwargs_intraday = dict(end=session, days=2)
df_intraday = prices.get(**kwargs_intraday)
df_intraday

symbol                                            MSFT              \
                                                  open        high   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  278.910004  281.899994   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  280.760010  280.910004   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)  279.549988  280.339996   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)  279.869995  279.869995   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)  279.619995  279.785004   
...                                                ...         ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)  285.589996  286.109985   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)  285.929993  285.929993   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)  285.260010  285.380005   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)  285.234985  286.170013   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)  285.309998  285.809998   

symbol                                                                         \
                                                   low       close     volume   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  278.910004  280.760010  1282050.0   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  279.549988  279.553802   550448.0   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)  279.200012  279.769989   532356.0   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)  279.010010  279.568909   379377.0   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)  278.799988  279.070007   332695.0   
...                                                ...         ...        ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)  285.579987  285.920013   354635.0   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)  285.190002  285.260010   351330.0   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)  284.750000  285.230011   382935.0   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)  285.234985  285.320007   697921.0   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)  284.500000  285.380005  1235980.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)     NaN  NaN NaN   NaN    NaN   
...                                            ...  ...  ..   ...    ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)     NaN  NaN NaN   NaN    NaN   

symbol                                     AZN.L                        
                                            open high low close volume  
[2022-04-18 09:30:00, 2022-04-18 09:35:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-18 09:35:00, 2022-04-18 09:40:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-18 09:40:00, 2022-04-18 09:45:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-18 09:45:00, 2022-04-18 09:50:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-18 09:50:00, 2022-04-18 09:55:00)   NaN  NaN NaN   NaN    NaN  
...                                          ...  ...  ..   ...    ...  
[2022-04-19 15:35:00, 2022-04-19 15:40:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-19 15:40:00, 2022-04-19 15:45:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-19 15:45:00, 2022-04-19 15:50:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-19 15:50:00, 2022-04-19 15:55:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-19 15:55:00, 2022-04-19 16:00:00)   NaN  NaN NaN   NaN    NaN  

[288 rows x 15 columns]

### `tzout`

The `tzout` parameter provides for setting the index timezone.

Indexes comprising dates, for example for daily or monthly data, are by default timezone-naive.

In [8]:
df_daily.pt.tz == None

True

It makes little sense for a date to have a timezone, although tzout does provide for setting dates to "UTC" should that be required.

In [9]:
df = prices.get(**kwargs_daily, tzout="UTC")
df[:1]

symbol                           MSFT                                      \
                                 open        high         low       close   
2022-04-05 00:00:00+00:00  313.269989  314.869995  309.869995  310.880005   

symbol                              9988.HK                          AZN.L  \
                             volume    open high low close volume     open   
2022-04-05 00:00:00+00:00  23156700     NaN  NaN NaN   NaN    NaN  10112.0   

symbol                                                                
                              high           low    close     volume  
2022-04-05 00:00:00+00:00  10350.0  10078.323242  10334.0  2529617.0

In [10]:
df.pt.tz

<UTC>

By default the timezone of intraday prices is set to the same timezone that the `tzin` parameter evaluates to (whether passed or not). If neither `tzin` or `lead_symbol` are passed then this will be the default timezone of the prices instance.

In [11]:
prices.tz_default == df_intraday.pt.tz

True

If `lead_symbol` is passed then `tzin` and hence `tzout` will default to the timezone associated with the lead symbol.

In [12]:
df = prices.get(**kwargs_intraday, lead_symbol="9988.HK")
df[:1]

symbol                                     MSFT                        \
                                           open high low close volume   
[2022-04-14 09:30:00, 2022-04-14 09:35:00)  NaN  NaN NaN   NaN    NaN   

symbol                                        9988.HK                   \
                                                 open  high        low   
[2022-04-14 09:30:00, 2022-04-14 09:35:00)  97.650002  98.0  95.699997   

symbol                                                       AZN.L           \
                                                close volume  open high low   
[2022-04-14 09:30:00, 2022-04-14 09:35:00)  95.699997    0.0   NaN  NaN NaN   

symbol                                                   
                                           close volume  
[2022-04-14 09:30:00, 2022-04-14 09:35:00)   NaN    NaN

In [13]:
df.pt.tz == prices.timezones["9988.HK"]

True

And if `tzin` is passed then that takes precedence and `tzout` defaults to the same timezone as `tzin`. In the following example `end` is passed as a tz-naive timestamp and `tzin` is passed to define the timezone that should be attributed to `end` as London.

In [14]:
xlon = prices.calendars["AZN.L"]
tz = prices.timezones["AZN.L"]
end = xlon.session_close(session).astimezone(tz).astimezone(None)
kwargs = {**kwargs_intraday, **{"end":end}}
print(kwargs, "\n", tz)  # for reference

{'end': Timestamp('2022-04-19 15:30:00'), 'days': 2} 
 Europe/London


In [15]:
df = prices.get(**kwargs, tzin=tz)
df[:1]

symbol                                            MSFT              \
                                                  open        high   
[2022-04-14 15:30:00, 2022-04-14 15:35:00)  283.670013  283.973389   

symbol                                                                        \
                                                   low       close    volume   
[2022-04-14 15:30:00, 2022-04-14 15:35:00)  283.339996  283.825012  238962.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2022-04-14 15:30:00, 2022-04-14 15:35:00)     NaN  NaN NaN   NaN    NaN   

symbol                                             AZN.L                    \
                                                    open     high      low   
[2022-04-14 15:30:00, 2022-04-14 15:35:00)  10523.799805  10526.0  10520.0   

symbol                                                       
                                              close  volume  
[2022-04-14 15:30:00, 2022-04-14 15:35:00)  10526.0  5684.0

In [16]:
df.pt.tz == tz

True

To define a non-default output timezone, simply pass `tzout`. `tzout` can be defined in the same ways as `tzin`:
* a string that's valid input to `pytz.timezone`.
* an instance of `pytz.timezone`.
* a string of a symbol.

Note that in each of the following examples the price data is the same, although the index timezone may change.

In [17]:
df = prices.get(**kwargs_intraday, tzout="UTC")
df[:1]

symbol                                            MSFT              \
                                                  open        high   
[2022-04-18 13:30:00, 2022-04-18 13:35:00)  278.910004  281.899994   

symbol                                                                        \
                                                   low      close     volume   
[2022-04-18 13:30:00, 2022-04-18 13:35:00)  278.910004  280.76001  1282050.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2022-04-18 13:30:00, 2022-04-18 13:35:00)     NaN  NaN NaN   NaN    NaN   

symbol                                     AZN.L                        
                                            open high low close volume  
[2022-04-18 13:30:00, 2022-04-18 13:35:00)   NaN  NaN NaN   NaN    NaN

In [18]:
df.pt.tz

<UTC>

In [19]:
tz = prices.timezones["9988.HK"]
print(f"{tz=}\n")  # for reference
df = prices.get(**kwargs_intraday, tzout=tz)
df[:1]

tz=<DstTzInfo 'Asia/Hong_Kong' LMT+7:37:00 STD>



symbol                                            MSFT              \
                                                  open        high   
[2022-04-18 21:30:00, 2022-04-18 21:35:00)  278.910004  281.899994   

symbol                                                                        \
                                                   low      close     volume   
[2022-04-18 21:30:00, 2022-04-18 21:35:00)  278.910004  280.76001  1282050.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2022-04-18 21:30:00, 2022-04-18 21:35:00)     NaN  NaN NaN   NaN    NaN   

symbol                                     AZN.L                        
                                            open high low close volume  
[2022-04-18 21:30:00, 2022-04-18 21:35:00)   NaN  NaN NaN   NaN    NaN

In [20]:
df.pt.tz

<DstTzInfo 'Asia/Hong_Kong' LMT+7:37:00 STD>

In [21]:
df = prices.get(**kwargs_intraday, tzout="9988.HK")
df[:1]

symbol                                            MSFT              \
                                                  open        high   
[2022-04-18 21:30:00, 2022-04-18 21:35:00)  278.910004  281.899994   

symbol                                                                        \
                                                   low      close     volume   
[2022-04-18 21:30:00, 2022-04-18 21:35:00)  278.910004  280.76001  1282050.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2022-04-18 21:30:00, 2022-04-18 21:35:00)     NaN  NaN NaN   NaN    NaN   

symbol                                     AZN.L                        
                                            open high low close volume  
[2022-04-18 21:30:00, 2022-04-18 21:35:00)   NaN  NaN NaN   NaN    NaN

In [22]:
df.pt.tz

<DstTzInfo 'Asia/Hong_Kong' LMT+7:37:00 STD>

Finally, in the event a composite calendar is returned comprising daily and intraday data then the output will always be forced to UTC.

In [23]:
df_comp = prices.get(start=start_H1_oob, end=end, tzin="9988.HK", composite=True)
df_comp

symbol                                            MSFT              \
                                                  open        high   
[2020-05-12, 2020-05-12)                    186.800003  187.039993   
[2020-05-13, 2020-05-13)                    182.550003  184.050003   
[2020-05-14, 2020-05-14)                    177.539993  180.690002   
[2020-05-15, 2020-05-15)                    179.059998  187.059998   
[2020-05-18, 2020-05-18)                    185.750000  186.199997   
...                                                ...         ...   
[2022-04-18 19:35:00, 2022-04-18 19:40:00)  279.290009  279.380005   
[2022-04-18 19:40:00, 2022-04-18 19:45:00)  279.149994  279.380005   
[2022-04-18 19:45:00, 2022-04-18 19:50:00)  279.130005  279.130005   
[2022-04-18 19:50:00, 2022-04-18 19:55:00)  278.600006  279.059998   
[2022-04-18 19:55:00, 2022-04-18 20:00:00)  279.440002  280.619995   

symbol                                                              \
                                                   low       close   
[2020-05-12, 2020-05-12)                    182.300003  182.509995   
[2020-05-13, 2020-05-13)                    176.539993  179.750000   
[2020-05-14, 2020-05-14)                    175.679993  180.529999   
[2020-05-15, 2020-05-15)                    177.000000  183.160004   
[2020-05-18, 2020-05-18)                    183.960007  184.910004   
...                                                ...         ...   
[2022-04-18 19:35:00, 2022-04-18 19:40:00)  278.920013  279.149994   
[2022-04-18 19:40:00, 2022-04-18 19:45:00)  278.880005  279.119995   
[2022-04-18 19:45:00, 2022-04-18 19:50:00)  278.489990  278.619995   
[2022-04-18 19:50:00, 2022-04-18 19:55:00)  278.339996  279.029999   
[2022-04-18 19:55:00, 2022-04-18 20:00:00)  279.399994  280.600006   

symbol                                                     9988.HK  \
                                                volume        open   
[2020-05-12, 2020-05-12)                    32038200.0  198.899994   
[2020-05-13, 2020-05-13)                    44711500.0  195.500000   
[2020-05-14, 2020-05-14)                    41873900.0  194.500000   
[2020-05-15, 2020-05-15)                    46610400.0  195.000000   
[2020-05-18, 2020-05-18)                    35264500.0  198.300003   
...                                                ...         ...   
[2022-04-18 19:35:00, 2022-04-18 19:40:00)    225411.0         NaN   
[2022-04-18 19:40:00, 2022-04-18 19:45:00)    238573.0         NaN   
[2022-04-18 19:45:00, 2022-04-18 19:50:00)    404372.0         NaN   
[2022-04-18 19:50:00, 2022-04-18 19:55:00)    420672.0         NaN   
[2022-04-18 19:55:00, 2022-04-18 20:00:00)    868407.0         NaN   

symbol                                                              \
                                                  high         low   
[2020-05-12, 2020-05-12)                    199.699997  197.199997   
[2020-05-13, 2020-05-13)                    197.399994  194.300003   
[2020-05-14, 2020-05-14)                    195.899994  194.100006   
[2020-05-15, 2020-05-15)                    197.100006  194.100006   
[2020-05-18, 2020-05-18)                    203.000000  197.399994   
...                                                ...         ...   
[2022-04-18 19:35:00, 2022-04-18 19:40:00)         NaN         NaN   
[2022-04-18 19:40:00, 2022-04-18 19:45:00)         NaN         NaN   
[2022-04-18 19:45:00, 2022-04-18 19:50:00)         NaN         NaN   
[2022-04-18 19:50:00, 2022-04-18 19:55:00)         NaN         NaN   
[2022-04-18 19:55:00, 2022-04-18 20:00:00)         NaN         NaN   

symbol                                                               AZN.L  \
                                                 close      volume    open   
[2020-05-12, 2020-05-12)                    199.699997  14037259.0  8690.0   
[2020-05-13, 2020-05-13)                    196.300003  26074457.0  8800.0   
[2020-05-14, 2020-05-14)                    

In [24]:
df_comp.pt.tz

<UTC>

### `fill`

For each symbol `get` will always return prices for every indice that the calendar associated with the symbol is open. Missing values will only appear in indices during which the calendar associated with the symbol was closed. It follows that it's only possible for a prices table to have misssing values if symbols are associated with different calendars with different session times, as is the case in this example...

In [25]:
df_intraday

symbol                                            MSFT              \
                                                  open        high   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  278.910004  281.899994   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  280.760010  280.910004   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)  279.549988  280.339996   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)  279.869995  279.869995   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)  279.619995  279.785004   
...                                                ...         ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)  285.589996  286.109985   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)  285.929993  285.929993   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)  285.260010  285.380005   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)  285.234985  286.170013   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)  285.309998  285.809998   

symbol                                                                         \
                                                   low       close     volume   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  278.910004  280.760010  1282050.0   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  279.549988  279.553802   550448.0   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)  279.200012  279.769989   532356.0   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)  279.010010  279.568909   379377.0   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)  278.799988  279.070007   332695.0   
...                                                ...         ...        ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)  285.579987  285.920013   354635.0   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)  285.190002  285.260010   351330.0   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)  284.750000  285.230011   382935.0   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)  285.234985  285.320007   697921.0   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)  284.500000  285.380005  1235980.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)     NaN  NaN NaN   NaN    NaN   
...                                            ...  ...  ..   ...    ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)     NaN  NaN NaN   NaN    NaN   

symbol                                     AZN.L                        
                                            open high low close volume  
[2022-04-18 09:30:00, 2022-04-18 09:35:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-18 09:35:00, 2022-04-18 09:40:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-18 09:40:00, 2022-04-18 09:45:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-18 09:45:00, 2022-04-18 09:50:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-18 09:50:00, 2022-04-18 09:55:00)   NaN  NaN NaN   NaN    NaN  
...                                          ...  ...  ..   ...    ...  
[2022-04-19 15:35:00, 2022-04-19 15:40:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-19 15:40:00, 2022-04-19 15:45:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-19 15:45:00, 2022-04-19 15:50:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-19 15:50:00, 2022-04-19 15:55:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-19 15:55:00, 2022-04-19 16:00:00)   NaN  NaN NaN   NaN    NaN  

[288 rows x 15 columns]

The `fill` parameter exists to optionally fill those missing values when a symbol's exchange was closed. The parameter can take the following values:
* **"ffill"** to fill 'open', 'close', 'high' and 'low' values with the last available close price.
* **"bfill"** to fill 'open', 'close', 'high' and 'low' values with the next available open price.
* **"both"** to first fill as "ffill" and then fill any missing values at the start of the table with "bfill".

The `volume` column will always be filled with 0.

In [26]:
prices.get(**kwargs_intraday, fill="ffill")

symbol                                            MSFT              \
                                                  open        high   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  278.910004  281.899994   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  280.760010  280.910004   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)  279.549988  280.339996   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)  279.869995  279.869995   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)  279.619995  279.785004   
...                                                ...         ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)  285.589996  286.109985   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)  285.929993  285.929993   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)  285.260010  285.380005   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)  285.234985  286.170013   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)  285.309998  285.809998   

symbol                                                                         \
                                                   low       close     volume   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  278.910004  280.760010  1282050.0   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  279.549988  279.553802   550448.0   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)  279.200012  279.769989   532356.0   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)  279.010010  279.568909   379377.0   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)  278.799988  279.070007   332695.0   
...                                                ...         ...        ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)  285.579987  285.920013   354635.0   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)  285.190002  285.260010   351330.0   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)  284.750000  285.230011   382935.0   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)  285.234985  285.320007   697921.0   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)  284.500000  285.380005  1235980.0   

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)        NaN        NaN        NaN   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)        NaN        NaN        NaN   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)        NaN        NaN        NaN   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)        NaN        NaN        NaN   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)        NaN        NaN        NaN   
...                                               ...        ...        ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)  91.400002  91.400002  91.400002   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)  91.400002  91.400002  91.400002   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)  91.400002  91.400002  91.400002   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)  91.400002  91.400002  91.400002   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)  91.400002  91.400002  91.400002   

symbol                                                          AZN.L  \
                                                close volume     open   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)        NaN    NaN      NaN   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)        NaN    NaN      NaN   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)        NaN    NaN      NaN   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)        NaN    NaN      NaN   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)        NaN    NaN      NaN   
...                                               ...    ...      ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)  91.400002    0.0  10506.0   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)  91.400002    0.0  10506.0   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)  91.400002    0.0  10506.0   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)  91.400002    0.0  10506.0   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)  91.400002    0.0  10506.0   

symbol                                                   

In [27]:
prices.get(**kwargs_intraday, fill="both")

symbol                                            MSFT              \
                                                  open        high   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  278.910004  281.899994   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  280.760010  280.910004   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)  279.549988  280.339996   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)  279.869995  279.869995   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)  279.619995  279.785004   
...                                                ...         ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)  285.589996  286.109985   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)  285.929993  285.929993   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)  285.260010  285.380005   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)  285.234985  286.170013   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)  285.309998  285.809998   

symbol                                                                         \
                                                   low       close     volume   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  278.910004  280.760010  1282050.0   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  279.549988  279.553802   550448.0   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)  279.200012  279.769989   532356.0   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)  279.010010  279.568909   379377.0   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)  278.799988  279.070007   332695.0   
...                                                ...         ...        ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)  285.579987  285.920013   354635.0   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)  285.190002  285.260010   351330.0   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)  284.750000  285.230011   382935.0   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)  285.234985  285.320007   697921.0   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)  284.500000  285.380005  1235980.0   

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  92.650002  92.650002  92.650002   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  92.650002  92.650002  92.650002   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)  92.650002  92.650002  92.650002   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)  92.650002  92.650002  92.650002   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)  92.650002  92.650002  92.650002   
...                                               ...        ...        ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)  91.400002  91.400002  91.400002   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)  91.400002  91.400002  91.400002   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)  91.400002  91.400002  91.400002   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)  91.400002  91.400002  91.400002   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)  91.400002  91.400002  91.400002   

symbol                                                          AZN.L  \
                                                close volume     open   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  92.650002    0.0  10452.0   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  92.650002    0.0  10452.0   
[2022-04-18 09:40:00, 2022-04-18 09:45:00)  92.650002    0.0  10452.0   
[2022-04-18 09:45:00, 2022-04-18 09:50:00)  92.650002    0.0  10452.0   
[2022-04-18 09:50:00, 2022-04-18 09:55:00)  92.650002    0.0  10452.0   
...                                               ...    ...      ...   
[2022-04-19 15:35:00, 2022-04-19 15:40:00)  91.400002    0.0  10506.0   
[2022-04-19 15:40:00, 2022-04-19 15:45:00)  91.400002    0.0  10506.0   
[2022-04-19 15:45:00, 2022-04-19 15:50:00)  91.400002    0.0  10506.0   
[2022-04-19 15:50:00, 2022-04-19 15:55:00)  91.400002    0.0  10506.0   
[2022-04-19 15:55:00, 2022-04-19 16:00:00)  91.400002    0.0  10506.0   

symbol                                                   

With daily data `fill` will fill values for sessions when an exchange was not open. Compare the following...

In [28]:
prices_alt = PricesYahoo("MSFT, AZN.L, 9988.HK", lead_symbol="AZN.L")
args = ("1D", "2021-12-24", "2021-12-29")
prices_alt.get(*args)

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2021-12-24         NaN         NaN         NaN         NaN         NaN   
2021-12-27  335.459991  342.480011  335.429993  342.450012  19947000.0   
2021-12-28  343.149994  343.809998  340.320007  341.250000  15661500.0   
2021-12-29  341.299988  344.299988  339.679993  341.950012  15042000.0   

symbol       AZN.L                                             9988.HK  \
              open    high          low   close     volume        open   
2021-12-24  8613.0  8676.0  8608.487305  8611.0   234238.0  113.000000   
2021-12-27     NaN     NaN          NaN     NaN        NaN         NaN   
2021-12-28     NaN     NaN          NaN     NaN        NaN  113.300003   
2021-12-29  8657.0  8720.0  8651.545898  8656.0  1406164.0  112.300003   

symbol                                                      
                  high         low       close      volume  
2021-12-24  113.000000  113.000000  113.000000         0.0  
2021-12-27         NaN         NaN         NaN         NaN  
2021-12-28  114.199997  112.300003  113.500000  22985751.0  
2021-12-29  112.300003  110.099998  110.599998  19990686.0

In [29]:
prices_alt.get(*args, fill="both")

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2021-12-24  335.459991  335.459991  335.459991  335.459991         0.0   
2021-12-27  335.459991  342.480011  335.429993  342.450012  19947000.0   
2021-12-28  343.149994  343.809998  340.320007  341.250000  15661500.0   
2021-12-29  341.299988  344.299988  339.679993  341.950012  15042000.0   

symbol       AZN.L                                             9988.HK  \
              open    high          low   close     volume        open   
2021-12-24  8613.0  8676.0  8608.487305  8611.0   234238.0  113.000000   
2021-12-27  8611.0  8611.0  8611.000000  8611.0        0.0  113.000000   
2021-12-28  8611.0  8611.0  8611.000000  8611.0        0.0  113.300003   
2021-12-29  8657.0  8720.0  8651.545898  8656.0  1406164.0  112.300003   

symbol                                                      
                  high         low       close      volume  
2021-12-24  113.000000  113.000000  113.000000         0.0  
2021-12-27  113.000000  113.000000  113.000000         0.0  
2021-12-28  114.199997  112.300003  113.500000  22985751.0  
2021-12-29  112.300003  110.099998  110.599998  19990686.0

Notice how the missing prices for MSFT on the 24th are filled backwards with the open price on the 27th, whilst the missing prices for AZN.L are filled fowards with the close price on the 24th.

To maximise the possibility of filling even the initial row forwards, you can always see if the prior indice had a value that could be carried forward... 

In [30]:
df = prices_alt.get(*args, add_a_row=True, fill="ffill")
df[1:]

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2021-12-24  334.690002  334.690002  334.690002  334.690002         0.0   
2021-12-27  335.459991  342.480011  335.429993  342.450012  19947000.0   
2021-12-28  343.149994  343.809998  340.320007  341.250000  15661500.0   
2021-12-29  341.299988  344.299988  339.679993  341.950012  15042000.0   

symbol       AZN.L                                             9988.HK  \
              open    high          low   close     volume        open   
2021-12-24  8613.0  8676.0  8608.487305  8611.0   234238.0  113.000000   
2021-12-27  8611.0  8611.0  8611.000000  8611.0        0.0  113.000000   
2021-12-28  8611.0  8611.0  8611.000000  8611.0        0.0  113.300003   
2021-12-29  8657.0  8720.0  8651.545898  8656.0  1406164.0  112.300003   

symbol                                                      
                  high         low       close      volume  
2021-12-24  113.000000  113.000000  113.000000         0.0  
2021-12-27  113.000000  113.000000  113.000000         0.0  
2021-12-28  114.199997  112.300003  113.500000  22985751.0  
2021-12-29  112.300003  110.099998  110.599998  19990686.0

### `include` and `exclude`

* `include` takes a symbol or list of symbols to include in the returned table.
* `exclude` takes a symbol or list of symbols to exclude from the returned table.

Only one or neither of `include` and `exclude` can be passed.

In [31]:
prices.get(**kwargs_intraday, include="MSFT")[:2]

symbol                                            MSFT              \
                                                  open        high   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  278.910004  281.899994   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  280.760010  280.910004   

symbol                                                                         
                                                   low       close     volume  
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  278.910004  280.760010  1282050.0  
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  279.549988  279.553802   550448.0

In [32]:
prices.get(**kwargs_daily, exclude=["MSFT", "AZN.L"])[:2]

symbol         9988.HK                                                
                  open        high         low       close      volume
2022-04-05         NaN         NaN         NaN         NaN         NaN
2022-04-06  109.099998  109.800003  107.099998  107.599998  46638063.0

**NOTE**: When passing `include` or `exclude` it's important to recall that they are applied as a post-processing operation after the table has been created. Excluding a symbol will consequently have no affect on, for example, the evaluation of a final indice where the period end evalutes as an unaligned session close, which will still be evaluated as if prices included all symbols (see [openend](./anchor.ipynb#openend) section of [anchor](./anchor.ipynb) tutorial for explanation of how such final indices are evaluated).

### `side`

`side` can be passed to index a price table with a `pd.DateTimeIndex` when it would otherwise be indexed with a `pd.IntervalIndex`.

`side` can optionally take either "left" or "right" to define each indice as the, respectively, left or right side of the period that the indice represents.

In [33]:
df_intraday[:2]  # for comparison

symbol                                            MSFT              \
                                                  open        high   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  278.910004  281.899994   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  280.760010  280.910004   

symbol                                                                         \
                                                   low       close     volume   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)  278.910004  280.760010  1282050.0   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)  279.549988  279.553802   550448.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2022-04-18 09:30:00, 2022-04-18 09:35:00)     NaN  NaN NaN   NaN    NaN   
[2022-04-18 09:35:00, 2022-04-18 09:40:00)     NaN  NaN NaN   NaN    NaN   

symbol                                     AZN.L                        
                                            open high low close volume  
[2022-04-18 09:30:00, 2022-04-18 09:35:00)   NaN  NaN NaN   NaN    NaN  
[2022-04-18 09:35:00, 2022-04-18 09:40:00)   NaN  NaN NaN   NaN    NaN

In [34]:
prices.get(**kwargs_intraday, side="left")[:2]

symbol                           MSFT                                      \
                                 open        high         low       close   
2022-04-18 09:30:00-04:00  278.910004  281.899994  278.910004  280.760010   
2022-04-18 09:35:00-04:00  280.760010  280.910004  279.549988  279.553802   

symbol                               9988.HK                       AZN.L       \
                              volume    open high low close volume  open high   
2022-04-18 09:30:00-04:00  1282050.0     NaN  NaN NaN   NaN    NaN   NaN  NaN   
2022-04-18 09:35:00-04:00   550448.0     NaN  NaN NaN   NaN    NaN   NaN  NaN   

symbol                                      
                          low close volume  
2022-04-18 09:30:00-04:00 NaN   NaN    NaN  
2022-04-18 09:35:00-04:00 NaN   NaN    NaN

In [35]:
df = prices.get(**kwargs_intraday, side="right")
df[:2]

symbol                           MSFT                                      \
                                 open        high         low       close   
2022-04-18 09:35:00-04:00  278.910004  281.899994  278.910004  280.760010   
2022-04-18 09:40:00-04:00  280.760010  280.910004  279.549988  279.553802   

symbol                               9988.HK                       AZN.L       \
                              volume    open high low close volume  open high   
2022-04-18 09:35:00-04:00  1282050.0     NaN  NaN NaN   NaN    NaN   NaN  NaN   
2022-04-18 09:40:00-04:00   550448.0     NaN  NaN NaN   NaN    NaN   NaN  NaN   

symbol                                      
                          low close volume  
2022-04-18 09:35:00-04:00 NaN   NaN    NaN  
2022-04-18 09:40:00-04:00 NaN   NaN    NaN

NOTE: If side is passed the the returned prices table will not have access to the `.pt` accessor.

In [ ]:
df.pt

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-47-8d21b9bf0d0d> in <module>
----> 1 df.pt

ValueError: PT accessor not available where index is a pd.DatatimeIndex with one or more indices that have a time component (Index must be pd.IntervalIndex if any indice includes a time component).
```

### `close_only`

`close_only` can be passed as `True` to return only the close price columns for each symbol.

In [37]:
prices.get(**kwargs_daily, close_only=True)[:4]

symbol,MSFT,9988.HK,AZN.L
2022-04-05,310.880005,NaN,10334.0
2022-04-06,299.500000,107.599998,10462.0
2022-04-07,301.369995,105.199997,10668.0
2022-04-08,296.970001,103.800003,10930.0


NOTE: if `close_only` is passed as True then the returned price table will not have access to the `.pt` accessor.

### `lose_single_symbol`

**`lose_single_symbol`** is only relevant if the table comprises a single symbol, in which case can be used to remove the symbol level of the columns index.

In [38]:
prices_us = PricesYahoo("MSFT")

In [39]:
prices_us.get(**kwargs_intraday)[:3]

symbol                                            MSFT              \
                                                  open        high   
[2022-04-18 09:30:00, 2022-04-18 10:30:00)  278.910004  282.390015   
[2022-04-18 10:30:00, 2022-04-18 11:30:00)  281.989990  282.459991   
[2022-04-18 11:30:00, 2022-04-18 12:30:00)  280.010010  280.160004   

symbol                                                                       
                                                   low       close   volume  
[2022-04-18 09:30:00, 2022-04-18 10:30:00)  278.790009  281.989990  5547464  
[2022-04-18 10:30:00, 2022-04-18 11:30:00)  279.750000  280.010010  2311993  
[2022-04-18 11:30:00, 2022-04-18 12:30:00)  278.750000  278.799988  2029667

In [40]:
prices_us.get(**kwargs_intraday, lose_single_symbol=True)[:3]

,open,high,low,close,volume
"[2022-04-18 09:30:00, 2022-04-18 10:30:00)",278.910004,282.390015,278.790009,281.989990,5547464
"[2022-04-18 10:30:00, 2022-04-18 11:30:00)",281.989990,282.459991,279.750000,280.010010,2311993
"[2022-04-18 11:30:00, 2022-04-18 12:30:00)",280.010010,280.160004,278.750000,278.799988,2029667


NOTE: `lose_single_symbol` is ignored if `close_only` is passed as `True`.